In [1]:
## Setting the PySpark environment variables

# import os

# os.environ['SPARK_HOME'] = "/opt/spark/spark-3.5.1-bin-hadoop3"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
# os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'

In [2]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import desc

In [3]:
# Create a SparkSession object

spark = SparkSession.builder.appName("DataFrame-Demo").getOrCreate()

24/03/13 13:01:52 WARN Utils: Your hostname, dhiraj resolves to a loopback address: 127.0.1.1; using 192.168.10.66 instead (on interface wlo1)
24/03/13 13:01:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/13 13:01:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/13 13:01:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Reading data from sources

### Using Dataframe

In [4]:
# reading data from /data/data.txt file
df = spark.read.text("../data/text/word_count.txt")

# appllying groupBy(), count() and orderBY()
result_df = df.selectExpr("explode(split(value, ' ')) as word") \
    .groupBy("word").count().orderBy(desc("count"))

In [5]:
# access top 10 data/count

result_df.take(10)

[Row(word='the', count=12),
 Row(word='of', count=7),
 Row(word='a', count=7),
 Row(word='in', count=5),
 Row(word='distributed', count=5),
 Row(word='Spark', count=4),
 Row(word='API', count=3),
 Row(word='RDD', count=3),
 Row(word='is', count=3),
 Row(word='on', count=3)]

### Using RDD

In [6]:
rdd = spark.sparkContext.textFile("../data/text/word_count.txt")

# reading data from source & manipulation is tough using RDD
result_rdd = rdd.flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda x: x[1], ascending=False)

In [7]:
result_rdd.take(10)

[('the', 12),
 ('of', 7),
 ('a', 7),
 ('in', 5),
 ('distributed', 5),
 ('Spark', 4),
 ('is', 3),
 ('as', 3),
 ('API', 3),
 ('on', 3)]

In [8]:
# Stopping spark session
spark.stop()